In [1]:
import numpy as np
from scipy.spatial.transform import Rotation
import meshplot as mp
import triangle as tr
import meshio
import itertools
import igl
import ipctk

In [2]:
# cmesh = ipctk.CollisionMesh(V_irregular, ipctk.edges(F_irregular), F_irregular)
# ipctk.has_intersections(cmesh, V_irregular)

In [3]:
shading = {
    "flat": True, # Flat or smooth shading of triangles
    "wireframe": True, "wire_width": 0.1, "wire_color": "black", # Wireframe rendering
    "width": 600, "height": 600, # Size of the viewer canvas
    "antialias": True, # Antialising, might not work on all GPUs
    "scale": 2.0, # Scaling of the model
    "side": "DoubleSide", # FrontSide, BackSide or DoubleSide rendering of the triangles
    "colormap": "viridis", "normalize": [None, None], # Colormap and normalization for colors
    "background": "#ffffff", # Background color of the canvas
    "line_width": 2.0, "line_color": "black", # Line properties of overlay lines
    "bbox": False, # Enable plotting of bounding box
    "point_color": "red", "point_size": 0.05 # Point properties of overlay points
}

In [4]:
example_mesh_file = "../meshes/octocat/octocat-coarse.ply"
# example_mesh_file = "../meshes/cube/cube.ply"

In [5]:
mesh = meshio.read(example_mesh_file)
# plt = mp.plot(mesh.points, mesh.cells[0].data, shading=shading)

In [6]:
def stitch_mesh(V, F):
    out_V, indices, inverse, out_F = igl.remove_duplicate_vertices(V, F, epsilon=1e-5)
    # indices, arg_indices = np.sort(indices), np.argsort(indices)
    # for i, inv in enumerate(inverse):
    #     new_inv = np.where(inv == arg_indices)[0]
    #     assert(new_inv.size == 1)
    #     inverse[i] = new_inv[0]
    # return V[indices], np.hstack([inverse[F[:, j]][:, None] for j in range(F.shape[1])])
    return out_V, out_F

def edge_lengths(V, F):
    return np.linalg.norm(V[F[:, [1, 2, 0]]] - V[F[:, [0, 1, 2]]], axis=2)

def min_edge_length(V, F):
    return edge_lengths(V, F).min()

def max_edge_length(V, F):
    return edge_lengths(V, F).max()

def average_edge_length(V, F):
    return edge_lengths(V, F).mean()

def print_edge_length_stats(V, F):
    print("Min edge length: ", min_edge_length(V, F))
    print("Max edge length: ", max_edge_length(V, F))
    print("Average edge length: ", average_edge_length(V, F))

In [7]:
def regular_grid_triangle_barycentric_coordinates(n):
    delta = 1 / (n - 1)
    # map from (i, j) coordinates to vertex id
    ij2v = np.full((n, n), -1)
    V = []

    for i in range(n):
        for j in range (n - i):
            ij2v[i, j] = len(V)
            V.append([i * delta, j * delta])

    # Create triangulated faces
    F = []
    for i, j in itertools.product(range(n - 1), range(n - 1)):
        for f in [ij2v[[i, i + 1, i], [j, j, j + 1]],
                  ij2v[[i + 1, i + 1, i], [j, j + 1, j + 1]]]:
            if (f >= 0).all():
                F.append(f)

    return np.array(V), np.array(F)

def sample_triangle(a, b, c, coords):
    # c
    # | \
    # a--b
    V = np.empty((coords.shape[0], a.size), dtype="float")
    for i, (alpha, beta) in enumerate(coords):
        V[i] = alpha * (b - a) + beta * (c - a) + a
    return V

def regular_grid_tessilation(V, F, out_max_edge_length):
    out_V = []
    out_F = []

    # Add one because n is the number of edge vertices not edges segments
    n = max(1, int(np.ceil(max_edge_length(V, F) / out_max_edge_length))) + 1

    coords, local_F = regular_grid_triangle_barycentric_coordinates(n)

    for triangle in F:
        out_F.extend(local_F + len(out_V))
        out_V.extend(sample_triangle(*V[triangle], coords))
    
    return stitch_mesh(np.array(out_V), np.array(out_F))

In [8]:
V_grid, F_grid = regular_grid_tessilation(
    mesh.points, 
    mesh.cells[0].data, 
    0.1 * max_edge_length(mesh.points, mesh.cells[0].data))

In [9]:
print_edge_length_stats(mesh.points, mesh.cells[0].data)
print()
print_edge_length_stats(V_grid, F_grid)

Min edge length:  0.5331873026281754
Max edge length:  11.436348954385352
Average edge length:  2.847147187078301

Min edge length:  0.05331873026281627
Max edge length:  1.1436348954385367
Average edge length:  0.2847147187078301


In [19]:
mp.plot(V_grid, F_grid, shading=shading)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.030651…

In [11]:
regular_mesh = meshio.Mesh(points=V_grid, cells=[("triangle", F_grid)])
regular_mesh.write("/Users/zachary/Desktop/regular_tessilation.ply")

Warning: PLY doesn't support 64-bit integers. Casting down to 32-bit.

In [12]:
def refine_edge(a, b, max_edge_length):
    n = int(np.ceil(np.linalg.norm(b - a) / max_edge_length))
    return np.array([
        (b - a) * t + a
        for t in np.linspace(0.0, 1.0, n+1)
    ])


def refine_triangle_edges(a, b, c, max_edge_len):
    out_V = np.vstack([
        refine_edge(a, b, max_edge_len)[:-1],
        refine_edge(b, c, max_edge_len)[:-1],
        refine_edge(c, a, max_edge_len)[:-1],
    ])
    out_E = np.vstack([
        np.hstack([
            np.arange(0, out_V.shape[0] - 1, dtype=int).reshape(-1, 1),
            np.arange(1, out_V.shape[0],     dtype=int).reshape(-1, 1),
        ]),
        [out_V.shape[0] - 1, 0]
    ])
    return out_V, out_E


def triangle_area(a, b, c):
    return np.linalg.norm(np.cross(b - a, c - a)) / 2

def irregular_triangle(a, b, c, max_edge_length):
    p = 3 * max_edge_length / 2
    max_area = np.sqrt(p * (p - max_edge_length)**3)

    V, E = refine_triangle_edges(a, b, c, max_edge_length)

    # Compute a rotation that aligns the z axis with the triangle normal
    R = Rotation.align_vectors(
        np.array([[0, 0, 1]]),
        np.cross(b - a, c - a).reshape(1, 3))[0].as_matrix()

    V_2D = V @ R.T # Align the triangle with the z axis
    z = V_2D[:, 2].copy()
    assert((abs(z - z[0]) < 1e-12).all())
    z = z[0] # Save the z-offset
    V_2D = V_2D[:, :2] # Drop the z coordinate
    
    t = tr.triangulate({"vertices": V_2D, "segments": E}, f'Ya{max_area:f}q')

    V = np.hstack([
        t["vertices"], 
        np.full((t["vertices"].shape[0], 1), z) # Restore the z-offset
    ]) @ R # Rotate back to the original orientation
    F = t["triangles"]

    # TODO: IDK why there are zero-area faces
    F = F[[triangle_area(*V[f]) > 1e-12 for f in F]]

    return V, F


def irregular_tessilation(V, F, max_edge_length):
    out_V = []
    out_F = []

    for triangle in F:
        local_V, local_F = irregular_triangle(*V[triangle], max_edge_length)
        out_F.extend(local_F + len(out_V))
        out_V.extend(local_V)
    
    return stitch_mesh(np.array(out_V), np.array(out_F))

In [13]:
V_irregular, F_irregular = irregular_tessilation(
    mesh.points,
    mesh.cells[0].data,
    0.125 * max_edge_length(mesh.points, mesh.cells[0].data))

/var/folders/y1/l7pg454j0jv6b028zqg1frsc0000gn/T/ipykernel_9312/1067447991.py:35: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  R = Rotation.align_vectors(


In [14]:
print_edge_length_stats(mesh.points, mesh.cells[0].data)
print()
print_edge_length_stats(V_grid, F_grid)
print()
print_edge_length_stats(V_irregular, F_irregular)

Min edge length:  0.5331873026281754
Max edge length:  11.436348954385352
Average edge length:  2.847147187078301

Min edge length:  0.05331873026281627
Max edge length:  1.1436348954385367
Average edge length:  0.2847147187078301

Min edge length:  0.22763592055667953
Max edge length:  1.9559550525231664
Average edge length:  1.1113467343545431


In [15]:
mp.plot(V_irregular, F_irregular, shading=shading)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.030651…

In [16]:
irregular_mesh = meshio.Mesh(points=V_irregular, cells=[("triangle", F_irregular)])
irregular_mesh.write("/Users/zachary/Desktop/irregular_tessilation.ply")

In [17]:
import trimesh

In [18]:
tmesh = trimesh.Trimesh(V_irregular, F_irregular)
print(len(trimesh.repair.broken_faces(tmesh)))
# trimesh.repair.fix_winding(mesh)
# print(len(trimesh.repair.broken_faces(mesh)))

0
